<a href="https://colab.research.google.com/github/DavideScassola/PML2024/blob/main/Notebooks/03_pyro_basics_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook 3: **Pyro** basics exercises

In [1]:
!pip install pyro-ppl

    opencv-python (>=3.) ; extra == 'all'
                  ~~~~^
ERROR: Exception:
Traceback (most recent call last):
  File "c:\users\ahmed tarek\anaconda3\lib\site-packages\pip\_internal\cli\base_command.py", line 105, in _run_wrapper
    status = _inner_run()
  File "c:\users\ahmed tarek\anaconda3\lib\site-packages\pip\_internal\cli\base_command.py", line 96, in _inner_run
    return self.run(options, args)
  File "c:\users\ahmed tarek\anaconda3\lib\site-packages\pip\_internal\cli\req_command.py", line 67, in wrapper
    return func(self, options, args)
  File "c:\users\ahmed tarek\anaconda3\lib\site-packages\pip\_internal\commands\install.py", line 483, in run
    installed_versions[distribution.canonical_name] = distribution.version
  File "c:\users\ahmed tarek\anaconda3\lib\site-packages\pip\_internal\metadata\pkg_resources.py", line 192, in version
    return parse_version(self._dist.version)
  File "c:\users\ahmed tarek\anaconda3\lib\site-packages\pip\_vendor\packaging\version

   --------------------------------------- 756.0/756.0 kB 10.5 MB/s eta 0:00:00
   --------------------------------------- 199.4/199.4 MB 26.0 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.10.2
    Uninstalling torch-1.10.2:
      Successfully uninstalled torch-1.10.2


    raise InvalidVersion(f"Invalid version: '{version}'")
pip._vendor.packaging.version.InvalidVersion: Invalid version: '4.0.0-unsupported'


In [2]:
import torch
import seaborn as sns
import matplotlib.pyplot as plt

import pyro
import pyro.distributions as dist

### Exercise 1

Consider the following model of the delay in getting to the workplace:

$$A\text{(alarm clock did not ring)} \sim Bern(0.1)$$

$$R\text{(heavy rain)} \sim Bern(0.15)$$

$$T\text{(traffic jam delay)} \sim Exponential(\mu = 5 + 10R) \ \ (\text{minutes})$$

$$D\text{(total delay)} \sim N(\mu = T + 30A, \sigma = 5) \ \ (\text{minutes})$$

1. Write the corresponding model in pyro
2. plot the corersponding graphical model
3. Sample 25000 samples from $p(A,R,T,D)$
4. Estimate $p(D)$ from the simulated values
5. Estimate $p(D | A)$ from the simulated values
6. Estimate $p(A | D>30m)$ from the simulated values

IMPORTANT: notice that `dist.Exponential` takes the rate as argument and not the mean

#### Solution

In [3]:
def delay_workspace():
    A = pyro.sample("A", dist.Bernoulli(0.1))
    alarm = "Alarm ring" if A.item() == 1.0 else "Alarm didn't ring"

    R = pyro.sample("R", dist.Bernoulli(0.15))
    rain = "Heavy rain" if R.item() == 1.5 else "not heavy rain"

    mu_T = 5 + 10*R
    T = pyro.sample("T", dist.Exponential(mu_T))

    mu_D = T + 30*A
    D = pyro.sample("D", dist.Normal(loc=mu_D,scale=5))


    print(f"{alarm} exam + {rain} ,  Traffic jam delay & Total delay = {T,D}")

for _ in range(2500):
    delay_workspace()

TypeError: __init__() got an unexpected keyword argument 'loc'

### Exercise 2

Do you remember this exercise?

*You are handed a fair coin with probability $0.5$ or an unfair coin (having $P(\text{head})=0.8$) with probability $0.5$. Then you toss it two times, with results $H_1$ and $H_2$.*

*Let's call $C$ the random variable describing if the coin is fair or not.*

Now you can give an approximate answer to the following questions through simulation with pyro:

1. Compute $p(h_1)$
2. Compute $p(c | h_1)$
3. Compute $p(h_2 | h_1)$

#### Solution

### Exercise 3

You have a set of $N$ observations $\text{height}_i$ of height of $N$ individuals all coming from an unknown country and you want to build a model of it. 
A (pretty simplistic) model is:
$$\mu_{male} \sim \mathcal{N}(\text{loc}=177, \text{scale}=5) \text{   (cm)}$$
$$\mu_{female} \sim \mathcal{N}(\text{loc}=164, \text{scale}=5)  \text{   (cm)}$$
$$S_i (\text{sex}) \sim Bern(0.5)$$
$$H_i (\text{height}) \sim \mathcal{N}(  \text{loc}=\mu,   \text{scale}= \frac{ \mu}{40}) \text{ where } \mu = \mu_{male} \text{ if } S_i=\text{male else } \mu_{female}  \text{   (cm)}$$

1. Write the corresponding model in pyro (assuming you actually have some observations).
2. Plot the corresponding graphical model.
3. Fixing $N=10000$, sample from the model, show the sampled values of $\mu_{male}$ and $\mu_{female}$ and plot the histogram of $H_i$. Do it two times (for two different samples).
4. Fixing $N=1$, estimate $p(\text{height})$ by simulation (you can redefine the model if it is simpler for you).
5. Fixing $N=1$, estimate $P(S = \text{male}| \text{height}> 180)$ by simulation.

#### Solution

### Exercise 4

It seems there is a correlation between the average weight of an animal species and its average longevity.

<img src="https://qph.cf2.quoracdn.net/main-qimg-427d1e41011ad526b9b186daf661bb54-pjlq" alt="drawing" width="500"/>

Suppose you have a dataset of $N$ observations $(\text{weight}_i, \text{longevity}_i)$ relative to different species and that you are only intersted in $p(\text{longevity} | \text{weight})$ (so you are not intersted in $p(\text{weight})$).

Given the following linear model:
$$p(\text{longevity} | \text{weight}, \alpha, h, \beta) = \mathcal{N}(\text{longevity}; \text{loc} =  h + \alpha \cdot \text{weight}; \text{scale} = \beta) $$

where

$$p(\alpha) = \mathcal{N}(\alpha; \text{loc} = 0, \text{scale} = 0.15) $$
$$p(h) = \mathcal{N}(h; \text{loc} = 0, \text{scale} = 0.15) $$
$$p(\beta) = Exponential(\beta; \mu = 0.2) $$



where longevity is in $\log_{10}$ years and weight is in $\log_{10}$ grams.

1. Define the model in pyro supposing to have the following observed data and plot the corresponding graphical model.

`observations = [(1.05, -0.3), (3.1, 0.84), (5.17, 2.1)]`

2. Now suppose you don't have any observation, estimate $p(\text{longevity} | \text{weight=100g})$ and $p(\text{longevity} | \text{weight=1000kg})$ by simulation.
3. Now suppose someone fitted a bayesian model given some data, and found that approximately:

    $$p(\alpha) = \mathcal{N}(\alpha; \text{loc} = 0.2, \text{scale} = 0.025) $$
    $$p(h) = \mathcal{N}(h; \text{loc} = 0.2, \text{scale} = 0.02) $$
    $$\beta \approx 0.05 \text{  (so this parameter was not fitted in a bayesian way)}$$

    Given this new knowledge, estimate $p(\text{longevity} | \text{weight=100g})$ and $p(\text{longevity} | \text{weight=1000kg})$ by simulation.

#### Solution